In [ ]:
import sys
import snappy
from snappy import ProductIO, GPF, ProductUtils, ProgressMonitor
import numpy
from osgeo import gdal
from PIL import Image
from jupytep.common.io import *
from jupytep.maps.leaflet.layer import ImageLayer
from jupytep.maps.mapnik.renders import RenderMap
from jupytep.maps.geoserver.loader import *
from jupytep.maps.content import *

In [ ]:
jpy = snappy.jpy

# More Java type definitions required for image generation
Color = jpy.get_type('java.awt.Color')
ColorPoint = jpy.get_type('org.esa.snap.core.datamodel.ColorPaletteDef$Point')
ColorPaletteDef = jpy.get_type('org.esa.snap.core.datamodel.ColorPaletteDef')
ImageInfo = jpy.get_type('org.esa.snap.core.datamodel.ImageInfo')
ImageLegend = jpy.get_type('org.esa.snap.core.datamodel.ImageLegend')
ImageManager = jpy.get_type('org.esa.snap.core.image.ImageManager')
JAI = jpy.get_type('javax.media.jai.JAI')
RenderedImage = jpy.get_type('java.awt.image.RenderedImage')

# Disable JAI native MediaLib extensions 
System = jpy.get_type('java.lang.System')
System.setProperty('com.sun.media.jai.disableMediaLib', 'true')

In [ ]:
def write_rgb_image(bands, filename, format):
    image_info = ProductUtils.createImageInfo(bands, True, ProgressMonitor.NULL)
    im = ImageManager.getInstance().createColoredBandImage(bands, image_info, 0)
    JAI.create("filestore", im, filename, format)

In [ ]:
file = '/eodata/Sentinel-2/MSI/L1C/2018/05/21/S2B_MSIL1C_20180521T100029_N0206_R122_T33UXT_20180521T120149.SAFE'
product = ProductIO.readProduct(file)

In [ ]:
red = product.getBand('B4')
green = product.getBand('B3')
blue = product.getBand('B2')

output_file = get_work_dir() + 'snappy_write_S2_rgb.png'
image_format = 'PNG'
write_rgb_image([red, green, blue], output_file, image_format)

In [ ]:
im = Image.open(output_file)
nx, ny = im.size
im2 = im.resize((int(nx*0.2), int(ny*0.2)), Image.BICUBIC)
output_file_rgb_res = get_work_dir() + 'snappy_write_S2_rgb_res.png'
im2.save(output_file_rgb_res)

In [ ]:
layer=ImageLayer()
box=layer.get_bbox(file)
print(box)
layer.add_image_layer("../../../work/snappy_write_S2_rgb_res.png", box, 'S2_RGB_RES')
layer.show_layer()